In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import random

In [ ]:
with open('../dataset_full.json', 'r') as f:
    dataset = json.load(f)

In [ ]:
def get_distribution(dataset):
    distribution = {}
    abstract_lengths = []
    for k, v in dataset.items():
        for task in v['tasks']:
            distribution[task] = distribution.get(task, 0) + 1
        abstract_lengths.append(len(v['abstract']))
    df = pd.DataFrame.from_dict(distribution, orient='index')
    df = df.sort_values(0, ascending=False)
    df.columns=['count']
    return df

In [ ]:
df = pd.DataFrame.from_dict(distribution, orient='index')
df = df.sort_values(0, ascending=False)
df.columns=['count']
kept_classes = list(df.query('count > 100').index)

In [ ]:
dataset_train = {}
dataset_val = {}
dataset_test = {}

train_distrib = {}
val_distrib = {}
test_distrib = {}

# I want at least 10 items in test, 20, in validation, and the rest in training
# because I have a minimum of 100 sample for the lowest class, ie 70/20/10 split
items = list(dataset.items())
random.shuffle(items)
for k, v in items:
    keep_datapoint = False
    kept_tasks = []
    for task in v['tasks']:
        if task in kept_classes:
            keep_datapoint = True
            kept_tasks.append(task)
            
    
    if keep_datapoint:
        test_count = 10
        val_count = 20
        for task in kept_tasks:
            val_count = min(val_count, val_distrib.get(task, 0))
            test_count = min(test_count, test_distrib.get(task, 0))


        if test_count < 10 :
            for task in kept_tasks:
                test_distrib[task] = test_distrib.get(task,0) + 1
            dataset_test[k] = v
            continue
        elif val_count < 20:
            for task in kept_tasks:
                val_distrib[task] = val_distrib.get(task,0) + 1
            dataset_val[k] = v
            continue
        else:
            for task in kept_tasks:
                train_distrib[task] = train_distrib.get(task,0) + 1
            dataset_train[k] = v

    

In [ ]:
print(f' Train set : {len(dataset_train)} , Validation set : {len(dataset_val)}, Test set : {len(dataset_test)}')

In [ ]:
get_distribution(dataset_train).query('index in @kept_classes').plot.bar(figsize=(30,5))
plt.savefig('../img/Train_set_distribution')

In [ ]:
get_distribution(dataset_val).query('index in @kept_classes').plot.bar(figsize=(30,5))
plt.savefig('../img/Validation_set_distribution')

In [ ]:
get_distribution(dataset_test).query('index in @kept_classes').plot.bar(figsize=(30,5))
plt.savefig('../img/Test_set_distribution')

In [ ]:
with open('../train_split.json', 'w') as f:
    json.dump(dataset_train, f)

In [ ]:
with open('../test_split.json', 'w') as f:
    json.dump(dataset_test, f)

In [ ]:
with open('../val_split.json', 'w') as f:
    json.dump(dataset_val, f)